# Guide to tHMM

In [1]:
import numpy as np
import scipy.stats as sp

## Synthesizing Cells (not required by the user)

In [2]:
from lineage.CellVar import CellVar as c
from lineage.CellVar import _double

In [3]:
 T = np.array([[1.0, 0.0],
               [0.0, 1.0]])
    
parent_state = 1
parent_cell = c(state=parent_state, left=None, right=None, parent=None, gen=1)
left_cell, right_cell = parent_cell._divide(T)

In [4]:
print(left_cell, '\n', parent_cell.left)

Generation: 2, State: 1, Observation: This cell has no observations to report. 
 Generation: 2, State: 1, Observation: This cell has no observations to report.


In [5]:
print(right_cell, '\n', parent_cell.right)

Generation: 2, State: 1, Observation: This cell has no observations to report. 
 Generation: 2, State: 1, Observation: This cell has no observations to report.


## Creating a synthetic lineage (required by the user)

In [6]:
from lineage.LineageTree import LineageTree
from lineage.StateDistribution import StateDistribution

### Creating a lineage and setting the full lineage (unpruned) as the one to be used

The required probabilities are those that define the tree and act of state switching. This process works by first creating a hidden tree of empty cells. Empty cells are those that have their states set but do not have any observations attached to them. We then draw as many observations from each state distribution and assign those observations to those cells. The $\pi$ and $T$ parameters are easy to define. The number of states is $k$. We require for $\pi$ a $k\times 1$ list of probabilities. These probabilities must add up to $1$ and they should be either in a $1$-dimensional list or a $1$-dimensional numpy array. The $T$ parameter should be a square numpy matrix of size $k\times k$. The rows are the states in which we are transitioning from and the columns are the states in which we are transitioning to. Each row of $T$ should sum to $1$. The columns need not sum to $1$. 

In [7]:
# pi: the initial probability vector
pi = np.array([0.8, 0.2])

# T: transition probability matrix
T = np.array([[0.7, 0.3],
              [0.4, 0.6]])

The emission matrix $E$ is a little more complicated to define because this is where the user has complete freedom in defining what type of observation they care about. In particular, the user has to first begin with defining what observation he or she will want in their cells in their synthetic images. For example, if one is observing kinematics or physics, they might want to use Gaaussian distribution observations. In defining the random variables, the user will pull from a Gaussian distribution based on the mean and standard deviation of the different states he or she picks. They can also utilize the Gaussian probability distribution to define the likelihood as well. Furthermore, they can build an analytical estimator for their state distributions that yield the parameter estimates when given a list of observations. Finally, the user can also define a prune rule, which is essentially a boolean function that inspects a cell's observations and returns `True` if the cell's subtree (all the cells that are related to the cell in question and are of older generation) is to be pruned or `False` if the cell is safe from pruning. In the Gaussian example, a user can remove a cell's subtree if its observation is higher or lower than some fixed value.

We have already built, as an example, and as bioengineers, a model that resembles lineage trees. In our synthetic model, our emissions are multivariate. This first emission is a Bernoulli observation, $0$ implying death and $1$ implying division. The second and third emissions are continuous and are from exponential and gamma distributions respectively. Though these can be thought of cell lifetime's or periods in a certain cell phase, we want the user to know that these values can really mean anything and they are completely free in choosing what the emissions and their values mean. We define ways to calculate random variables for these multivariate observations and likelihoods of an observations. We also provide as a prune rule, keeping with the cell analogy, that if a cell has a $0$ in its Bernoulli observation, then its subtree is pruned from the full lineage tree. Though this will obviously introduce bias into estimation, we keep both the full tree and the pruned tree in the lineage objects, in the case a user would like to see the effects of analyzing on one versus the other. 

Ultimately, $E$ is defined as a $k\times 1$ size list of state distribution objects. These distribution objects are rich in what they can already do, and a user can easily add more to their functionality. They only need to be instantiated by what parameters define that state's distribution.

In [8]:
# E: states are defined as StateDistribution objects
# State 0 parameters
state0 = 0
bern_p0 = 0.7
expon_scale_beta0 = 20
gamma_a0 = 5.0
gamma_scale0 = 1.0

# State 1 parameters
state1 = 1
bern_p1 = 0.99
expon_scale_beta1 = 80
gamma_a1 = 10.0
gamma_scale1 = 2.0

state_obj0 = StateDistribution(state0, bern_p0, expon_scale_beta0, gamma_a0, gamma_scale0)
state_obj1 = StateDistribution(state1, bern_p1, expon_scale_beta1, gamma_a1, gamma_scale1)

E = [state_obj0, state_obj1]

The final required parameters are more obvious. The first is the desired number of cells one would like in their full unpruned lineage tree. This can be any number. The lineage tree is built '_from left to right_'. What this means is that, we construct the binary tree by going to the left-most cell, dividing then walking through the generation. For example, if someone requested for

In [9]:
desired_num_cells = 2**8 - 1 
prune_boolean = False # To get the full tree

In [10]:
lineage1 = LineageTree(pi, T, E, desired_num_cells, prune_boolean)
print(lineage1)

This tree is NOT pruned. It is made of 2 states.
 For each state in this tree: 
 	 There are 144 cells of state 0, 
 	 There are 111 cells of state 1.
 This UNpruned tree has 255 cells in total


In [11]:
desired_num_cells = 2**9 -1 
prune_boolean = True # To get pruned tree

In [12]:
lineage2 = LineageTree(pi, T, E, desired_num_cells, prune_boolean)
print(lineage2)

This tree is pruned. It is made of 2 states.
 For each state in this tree: 
 	 There are 5 cells of state 0, 
 	 There are 3 cells of state 1.
 This pruned tree has 8 cells in total


In [13]:
from lineage.CellNode import CellNode
from lineage.BaumWelch import fit
from lineage.DownwardRecursion import get_root_gammas, get_nonroot_gammas
from lineage.Viterbi import get_leaf_deltas, get_nonleaf_deltas, Viterbi
from lineage.UpwardRecursion import get_leaf_Normalizing_Factors, get_leaf_betas, get_nonleaf_NF_and_betas
from lineage.tHMM import tHMM
from lineage.tHMM_utils import max_gen, get_gen, get_parents_for_level, getAccuracy, getAIC
from lineage.Lineage_utils import get_numLineages, init_Population, remove_singleton_lineages, remove_unfinished_cells
from lineage.Lineage_utils import generatePopulationWithTime as gpt
from lineage.plotting_utils import make_colormap_graph
from lineage.Analyze import Analyze
from lineage.Depth_Two_State_Lineage import Depth_Two_State_Lineage

ModuleNotFoundError: No module named 'lineage.CellNode'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

## Creating and analyzing our synthetic lineage

Variables subscripted or prefixed with `MAS` are referring to the first lineage in this heterogeneous lineage, while variables subscripted with `2` are referring to the second lineage. These two lineages are what comprise the _depth two state_ lineage.

In [ ]:
T_MAS = 400. # how long the first synthetic lineage runs for
T_2 = 250. # how long the second synthetic lineage runs for
experimentTime = T_MAS + T_2
MASinitCells = [1] # how many initial cells are in the first lineage (has to be one)
MASlocBern = [0.999] # the bernoulli parameter regarding the chance of dividing over dying for the master (first) lineage
MASbeta = [80] # the exponential parameter regarding how long a cell lives [hours]

initCells2 = [1] # how many initial cells are in the second lineage (has to be one)
locBern2 = [0.8] # the bernoulli parameter regarding the change of dividing over dying for the second lineage
beta2 = [20] # the exponential parameter regarding how long a cell lives [hours]

# In general, the above parameters are described as lists because one can have multiple groups of parameters
# to describe their population. For example, one can easily create a population with initCells = [10, 50, 3],
# locBern = [0.8, 0.9, 0.75], beta = [80, 20, 90] to create three distinct populations. That is to say,
# 10+50+3=63 lineages will be created, each tree following their own distribution parameters using the same index
# in the other lists. 

# To create these depth heterogenous lineages, we created the function below that can join one lineage to another
# and also perform the right house-keeping tasks to make sure that the lineages are sufficiently joined, such as
# making sure the correct parent-daughter links exist, the cells in the second lineage recognize the root cell of
# the master lineage tree, etc.

max_lin_length = 300 # for the purpose of visualizing data, we set restrictions on the maximum length and
min_lin_length = 5 # minimum length of the lineage generated
FOM='E' # this just tells the code that we are using an exponential Force of Mortality
X, newLineage, masterLineage, subLineage2 = Depth_Two_State_Lineage(T_MAS, MASinitCells, MASlocBern, T_2, initCells2, locBern2, FOM=FOM, betaExp=MASbeta, betaExp2=beta2)

X1 = remove_singleton_lineages(X) # this removes lineages that are just one cell as they seem to break under the analysus
x_new, end_time = remove_unfinished_cells(X1)
# X = remove_unfinished_cells(X) # this removes cells that lived past the experimental end time

numStates = 2 # need to tell our code how many states we expect to see
_, _, all_states, tHMMobj, _, _ = Analyze(x_new, numStates) # this function contains the bulk of our analysis code
# that is, it neatly runs our analysis steps in order

# Visualizing results

In [ ]:
getAccuracy(tHMMobj, all_states, verbose=True) # this function simply runs the state comparison and checks for accuracy
import matplotlib as mpl
from matplotlib import ticker

G, cmap, _ = make_colormap_graph(x_new)
M = G.number_of_edges()
edge_weights = [d for (u,v,d) in G.edges.data('weight')]
#pos prog options: neato, dot, twopi, circo (don't use), fdp (don't use), nop (don't use), wc (don't use), acyclic (don't use), gvpr (don't use), gvcolor (don't use), ccomps (don't use), sccmap (don't use), tred (don't use), sfdp (don't use), unflatten (don't use)
pos = graphviz_layout(G, prog='dot', root=0)
plt.figure(figsize=(40,31))
plt.figaspect(1)
node_size = 200
nodes = nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color=cmap, alpha=1)
edges = nx.draw_networkx_edges(G, pos, node_size=node_size, edge_color=edge_weights, edge_cmap=plt.cm.plasma, width=2)

ax = plt.gca()
ax.set_axis_off()
cb = plt.colorbar(edges, ticks=edge_weights)
tick_locator = ticker.MaxNLocator(nbins=5)
cb.locator = tick_locator
cb.update_ticks()
cb.set_label(label=r'Experiment Time [hrs]', labelpad=90)
cb.ax.invert_yaxis()
plt.title('Simulated Heterogeneous (by Depth) Lineage')
plt.rcParams.update({'font.size': 64})
plt.savefig('true.svg')
plt.show()

In [ ]:
G, cmap, _ = make_colormap_graph(x_new, tHMMobj.states[0])
M = G.number_of_edges()
edge_weights = [d for (u,v,d) in G.edges.data('weight')]
#pos prog options: neato, dot, twopi, circo (don't use), fdp (don't use), nop (don't use), wc (don't use), acyclic (don't use), gvpr (don't use), gvcolor (don't use), ccomps (don't use), sccmap (don't use), tred (don't use), sfdp (don't use), unflatten (don't use)
pos = graphviz_layout(G, prog='dot', root=0)
plt.figure(figsize=(40,31))
plt.figaspect(1)
node_size = 200
nodes = nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color=cmap, alpha=1)
edges = nx.draw_networkx_edges(G, pos, node_size=node_size, edge_color=edge_weights, edge_cmap=plt.cm.plasma, width=2)

ax = plt.gca()
ax.set_axis_off()
cb = plt.colorbar(edges, ticks=edge_weights)
tick_locator = ticker.MaxNLocator(nbins=5)
cb.locator = tick_locator
cb.update_ticks()
cb.set_label(label=r'Experiment Time [hrs]', labelpad=90)
cb.ax.invert_yaxis()
plt.title('Estimated Subpopulation Classification')
plt.rcParams.update({'font.size': 64})
plt.savefig('estimated.svg')
plt.show()